### Creating class for strategy

In [1]:
import pandas as pd
import numpy as np
import tpqoa
import time as timeout
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
api = tpqoa.tpqoa("oanda.cfg")

class SMAtrade(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, timeframe, SMA_S, SMA_L, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.timeframe = timeframe
        self.units = units
        self.data = None
        self.profits = [] # store p&l for all trades
        self.SMA_S = SMA_S
        self.SMA_L = SMA_L
    
    def prepare_data(self):
        try:
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            date_now = "{}-{}-{} {}:{}:{}".format(now.year, now.month, now.day, now.hour, now.minute, now.second)
            past = now - timedelta(days = 3) # three days of data is enough
            date_past = "{}-{}-{} {}:{}:{}".format(past.year, past.month, past.day, past.hour, past.minute, past.second)

            # downloading data
            df = self.get_history(instrument = instrument, start = date_past, end = date_now,
                                granularity = 'S5', price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            # resample data to our timeframe
            df = df.resample(self.bar_length, label="right").last().dropna().iloc[:-1]
            self.data = df.copy()
            # strategy specific
            self.data["SMA_S"] = self.data[self.instrument].rolling(self.SMA_S).mean()
            self.data["SMA_L"] = self.data[self.instrument].rolling(self.SMA_L).mean()
            self.data["position"] = np.where(self.data["SMA_S"] > self.data["SMA_L"], 1, -1)

            self.last_bar = self.data.index[-1] # change the data of last bar
        except:
                print("\nConnection error while downloading data, let's try again")
                timeout.sleep(3)
                pass
            
        return self.data
    
    # on_success function runs every_time there is a tick, so we need to make sure it is efficient
    def on_success(self, time, bid, ask):
        '''Method called when new data is retrieved. 
        '''
        try:
            if self.ticks % 2 == 0:
                print(self.ticks, end = " ")

                # prepare data & define strategy
                recent_tick = pd.to_datetime(time)
                if recent_tick - self.last_bar > self.bar_length:      
                    self.prepare_data()
                    print("\nNew candle added to dataset")
                    # executing trades
                    self.execute_trades()
                    # pause for half length of bar as nothing will happen then
                    timeout.sleep(250)
        except:
            print("\nConnection error while downloading tick data, let's try again")
            timeout.sleep(3)
            pass
            
    def execute_trades(self):
        global position
        try:
            #self.position = float(self.get_account_summary()['positionValue']) # read current position
            if self.data.position.iloc[-1] == 1 and self.data.position.iloc[-2] != self.data.position.iloc[-1]:
                if position == 0:
                    order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                    self.report_trade(order, "GOING LONG")
                    position = 1
                elif position < 0:
                    order = self.create_order(self.instrument, self.units*2, suppress = True, ret = True)
                    self.report_trade(order, "GOING LONG")
                    position = 1
            elif self.data.position.iloc[-1] == -1 and self.data.position.iloc[-2] != self.data.position.iloc[-1]:
                if position == 0:
                    order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                    self.report_trade(order, "GOING SHORT")
                    position = -1
                elif position > 0:
                    order = self.create_order(self.instrument, -self.units*2, suppress = True, ret = True)
                    self.report_trade(order, "GOING SHORT")
                    position = -1
        except:
            print("\nConnection error while executing trade, let's try again")
            timeout.sleep(3)
            pass
                
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  

In [2]:
units = 500
instrument = "NATGAS_USD"
timeframe = "M5"
bar_length = "5m"
sma_s = 32
sma_l = 190
position = 0

In [3]:
trader = SMAtrade("oanda.cfg", instrument, bar_length, timeframe, sma_s, sma_l, units)

In [4]:
trader.prepare_data()

,NATGAS_USD,SMA_S,SMA_L,position
time,,,,
2021-05-14 05:10:00+00:00,2.934,NaN,NaN,-1
2021-05-14 05:15:00+00:00,2.934,NaN,NaN,-1
2021-05-14 05:20:00+00:00,2.934,NaN,NaN,-1
2021-05-14 05:25:00+00:00,2.932,NaN,NaN,-1
2021-05-14 05:35:00+00:00,2.933,NaN,NaN,-1
...,...,...,...,...
2021-05-17 04:25:00+00:00,3.001,3.006813,2.957574,1
2021-05-17 04:30:00+00:00,3.001,3.006531,2.957905,1
2021-05-17 04:35:00+00:00,3.001,3.006250,2.958247,1


In [ ]:
while True:
    try:
        trader.stream_data(instrument)
    except:
        print("\nError running the function, let's try again")
        timeout.sleep(3)
        pass

2 
New candle added to dataset
4 
New candle added to dataset
6 
New candle added to dataset
8 10 12 14 16 
New candle added to dataset
18 20 22 24 
New candle added to dataset
26 
New candle added to dataset
28 
New candle added to dataset
30 
New candle added to dataset
32 
New candle added to dataset
34 
New candle added to dataset
36 38 
New candle added to dataset
40 
New candle added to dataset
42 
New candle added to dataset
44 
New candle added to dataset
46 
New candle added to dataset
48 50 
New candle added to dataset
52 54 
New candle added to dataset
56 58 60 
New candle added to dataset
62 
New candle added to dataset
64 66 68 
New candle added to dataset
70 72 74 76 78 
New candle added to dataset
80 82 84 86 
New candle added to dataset
88 90 
New candle added to dataset
92 94 96 
New candle added to dataset
98 
New candle added to dataset
100 102 104 106 108 110 112 
New candle added to dataset
114 116 
New candle added to dataset
118 120 
New candle added to dataset
1


New candle added to dataset
2008 
New candle added to dataset
2010 2012 2014 2016 2018 2020 
Error running the function, let's try again
2 
New candle added to dataset
4 6 8 10 12 14 16 18 20 22 
New candle added to dataset
24 26 28 30 32 34 36 38 
New candle added to dataset
40 42 44 46 48 50 52 54 
New candle added to dataset
56 58 60 62 64 
New candle added to dataset
66 68 70 72 74 76 78 80 82 
New candle added to dataset
84 86 88 90 92 94 96 98 100 102 
New candle added to dataset
104 106 108 110 112 114 116 
New candle added to dataset
118 120 122 124 126 128 130 132 134 136 138 
New candle added to dataset
140 142 144 146 148 150 152 154 156 158 160 
New candle added to dataset
162 164 166 168 170 
New candle added to dataset
172 174 176 178 180 182 184 186 188 190 192 194 196 198 200 
New candle added to dataset
202 
New candle added to dataset
204 206 208 210 212 214 216 218 220 222 
New candle added to dataset
224 226 
New candle added to dataset
228 230 232 234 236 238 240 

1438 1440 
New candle added to dataset
1442 1444 1446 
New candle added to dataset
1448 1450 
New candle added to dataset
1452 
New candle added to dataset
1454 1456 1458 1460 1462 
New candle added to dataset
1464 
New candle added to dataset
1466 
New candle added to dataset
1468 
New candle added to dataset
1470 1472 1474 1476 
New candle added to dataset
1478 1480 1482 
New candle added to dataset
1484 1486 1488 
New candle added to dataset
1490 1492 
New candle added to dataset
1494 1496 1498 1500 1502 1504 1506 
New candle added to dataset
1508 1510 1512 1514 1516 
New candle added to dataset
1518 1520 1522 1524 1526 1528 1530 1532 
New candle added to dataset
1534 1536 1538 1540 1542 
New candle added to dataset
1544 1546 1548 1550 1552 1554 1556 1558 1560 1562 
New candle added to dataset
1564 1566 1568 1570 1572 1574 1576 1578 1580 1582 1584 1586 
New candle added to dataset
1588 1590 1592 1594 1596 1598 1600 1602 1604 1606 1608 1610 
New candle added to dataset
1612 1614 1616

1562 
New candle added to dataset
1564 
New candle added to dataset
1566 1568 
New candle added to dataset
1570 
New candle added to dataset
1572 
New candle added to dataset
1574 
New candle added to dataset
1576 
New candle added to dataset
1578 1580 
New candle added to dataset
1582 1584 
New candle added to dataset
1586 1588 1590 1592 
New candle added to dataset
1594 
New candle added to dataset
1596 1598 1600 1602 
New candle added to dataset
1604 1606 
New candle added to dataset
1608 
New candle added to dataset
1610 1612 1614 1616 1618 1620 1622 
New candle added to dataset
1624 1626 
New candle added to dataset
1628 1630 1632 1634 
New candle added to dataset
1636 
New candle added to dataset
1638 
New candle added to dataset
1640 
New candle added to dataset
1642 
New candle added to dataset
1644 
New candle added to dataset
1646 
New candle added to dataset
1648 
New candle added to dataset
1650 
New candle added to dataset
1652 1654 1656 1658 1660 
New candle added to data

74 76 78 80 82 84 86 88 
New candle added to dataset
90 
New candle added to dataset
92 94 96 98 100 
New candle added to dataset
102 104 106 
New candle added to dataset
108 110 112 114 116 118 120 
New candle added to dataset
122 124 126 128 130 132 134 136 138 140 142 144 
New candle added to dataset
146 148 150 152 154 156 158 160 162 164 166 168 
New candle added to dataset
170 172 174 176 178 180 182 184 186 188 
New candle added to dataset
190 192 194 196 198 200 202 204 206 208 210 212 214 216 218 
New candle added to dataset
220 
New candle added to dataset
222 224 226 228 230 232 234 236 238 240 242 244 246 248 250 252 254 256 
New candle added to dataset
258 260 262 264 266 268 270 272 274 276 278 280 
New candle added to dataset
282 284 286 288 290 292 294 296 
New candle added to dataset
298 300 302 304 306 
New candle added to dataset
308 310 312 314 
New candle added to dataset
316 318 320 322 324 326 328 
New candle added to dataset
330 332 334 336 338 340 342 344 346 3

184 
New candle added to dataset
186 
New candle added to dataset
188 190 192 
New candle added to dataset
194 196 
New candle added to dataset
198 200 202 
New candle added to dataset
204 206 208 
New candle added to dataset
210 
New candle added to dataset
212 214 216 218 
New candle added to dataset
220 
New candle added to dataset
222 224 
New candle added to dataset
226 228 230 232 
New candle added to dataset
234 236 238 240 242 244 246 248 250 252 
New candle added to dataset
254 256 258 260 262 264 266 268 
New candle added to dataset
270 272 
New candle added to dataset
274 
New candle added to dataset
276 278 280 282 284 286 288 290 
New candle added to dataset
292 294 296 298 300 
New candle added to dataset
302 304 306 308 310 312 314 316 318 
Error running the function, let's try again
2 
New candle added to dataset
4 6 8 10 12 
New candle added to dataset
14 16 18 20 
New candle added to dataset
22 24 26 28 30 32 34 36 38 40 
New candle added to dataset
42 44 46 48 50 
Ne

434 
New candle added to dataset
436 
New candle added to dataset
438 
New candle added to dataset
440 
New candle added to dataset
442 444 446 
New candle added to dataset
448 
New candle added to dataset
450 
New candle added to dataset
452 
New candle added to dataset
454 
New candle added to dataset
456 458 
New candle added to dataset
460 
New candle added to dataset
462 
New candle added to dataset
464 466 468 470 
New candle added to dataset
472 474 
New candle added to dataset
476 
New candle added to dataset
478 
New candle added to dataset
480 
New candle added to dataset
482 484 
New candle added to dataset
486 
New candle added to dataset
488 
New candle added to dataset
490 
New candle added to dataset
492 
New candle added to dataset

----------------------------------------------------------------------------------------------------
2021-05-21T01:07:35.011235894Z | GOING LONG
2021-05-21T01:07:35.011235894Z | units = 1000.0 | price = 2.944 | P&L = -6.9829 | Cum P&L = 33.6

1156 1158 1160 1162 1164 1166 1168 
New candle added to dataset
1170 1172 1174 1176 1178 1180 
New candle added to dataset
1182 1184 1186 1188 1190 1192 1194 1196 
New candle added to dataset
1198 1200 1202 1204 1206 1208 1210 1212 1214 1216 1218 1220 
New candle added to dataset
1222 1224 1226 1228 1230 1232 1234 1236 1238 
New candle added to dataset
1240 1242 1244 1246 1248 1250 1252 1254 1256 1258 1260 
New candle added to dataset
1262 1264 1266 1268 1270 1272 1274 1276 
New candle added to dataset
1278 1280 1282 1284 1286 1288 1290 
New candle added to dataset
1292 1294 1296 1298 1300 1302 1304 
New candle added to dataset
1306 
New candle added to dataset
1308 1310 1312 1314 1316 
New candle added to dataset
1318 1320 1322 1324 1326 1328 1330 1332 1334 1336 1338 1340 1342 
New candle added to dataset
1344 1346 1348 1350 
New candle added to dataset
1352 1354 
New candle added to dataset
1356 1358 1360 1362 1364 1366 1368 1370 
New candle added to dataset
1372 1374 1376 1378 1380 

### sprawdzać odległość aktualnej ceny od ceny przecięcia, jeśli zbyt daleko to nie wchodzić w pozycję


In [ ]:
# show graph if required
trader.data.tail(30).plot(figsize = (12, 8), secondary_y = "position")
plt.show()